In [6]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron
iris=load_iris()
X=iris.data[:,(2,3)]
y=(iris.target==0).astype(np.int)
per_clf=Perceptron(random_state=42)
#相当于SGDclassifier(loss='perception',learning_rate='constant',eta0=1,penalty=None)
#不返回概率预测；以阈值为预测基础
per_clf.fit(X,y)

y_pred=per_clf.predict([[2,0.5]])
print(y_pred)

#ANN;MLP;BP是训练算法：对于每个样本，带人算法计算结果，根据结果和预测误差，修正权值
#FNN，前馈神经网络信号流只由一个方向传播：由输入到输出

#tf.contrib.learn tensorflow的高级封装
import tensorflow as tf
feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(X)
dnn_clf=tf.contrib.learn.DNNClassifier(hidden_units=[300,100],n_classes=10,
                                      feature_columns=feature_columns)
dnn_clf.fit(x=X,y=y,batch_size=50,steps=4000)
from sklearn.metrics import accuracy_score
y_pred=list(dnn_clf.predict(X))
accuracy_score(y,y_pred)
dnn_clf.evaluate(X,y)



[1]
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002E901B327F0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\hzg0601\\AppData\\Local\\Temp\\tmpbs2ujae9'}


D:\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\hzg0601\AppData\Local\Temp\tmpbs2ujae9\model.ckpt.
INFO:tensorflow:loss = 2.0945575, step = 1
INFO:tensorflow:global_step/sec: 499.746
INFO:tensorflow:loss = 0.01715524, step = 101 (0.202 sec)
INFO:tensorflow:global_step/sec: 661.214
INFO:tensorflow:loss = 0.0053736973, step = 201 (0.149 sec)
INFO:tensorflow:global_step/sec: 574.754
INFO:tensorflow:loss = 0.0025865587, step = 301 (0.175 sec)
INFO:tensorflow:global_step/sec: 697.71
INFO:tensorflow:loss = 0.0022533669, step = 401 (0.142 sec)
INFO:tensorflow:global_step/sec: 622.815
INFO:tensorflow:loss = 0.0026647553, step = 501 (0.162 sec)
INFO:tensorflow:global_step/sec: 638.14
INFO:tensorflow:loss = 0.000929228, step = 601 (0.156 sec)
INFO:tensorflow:global_step/sec: 733.178
INFO:tensorflow:loss = 0.0008848192, step = 7

{'loss': 0.00012087003, 'accuracy': 1.0, 'global_step': 4000}

In [16]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
n_inputs=28*28
n_hidden1=300
n_hidden2=100
n_outputs=10

X=tf.placeholder(tf.float32,shape=(None,n_inputs),name='X')#批的大小仍不确定
y=tf.placeholder(tf.int64,shape=(None),name='y')#每个样本1个，但根据批的大小而异

def neuron_layer(X,n_neurons,name,activation=None):
    with tf.name_scope(name):#创建以层名为名的命名域
        n_inputs=int(X.get_shape()[1])
        stddev=2/np.sqrt(n_inputs)#可以使网络快速收敛
        init=tf.truncated_normal((n_inputs,n_neurons),stddev=stddev)
        W=tf.Variable(init,name='weights')#初始化权值矩阵
        b=tf.Variable(tf.zeros([n_neurons]),name='biases')
        z=tf.matmul(X,W)+b
        if activation=='relu':
            return tf.nn.relu(z)
        else:
            return z
with tf.name_scope('dnn'):
    hidden1=neuron_layer(X,n_hidden1,'hidden1',activation='relu')
    hidden2=neuron_layer(hidden1,n_hidden2,'hidden2',activation='relu')
    logits=neuron_layer(hidden2,n_outputs,'outputs')#并不是输出，需要结合优化策略后再
    #进行输出层的设计
#tensorflow定义了完整的标准神经网络层，一般并不需要自己定义
from tensorflow.contrib.layers import fully_connected
with tf.name_scope('dnnfc'):
    hidden1=fully_connected(X,n_hidden1,scope='hidden1')
    hidden2=fully_connected(hidden1,n_hidden2,scope='hidden2')
    logits=fully_connected(hidden2,n_outputs,scope='outputs',activation_fn=None)
    #activation_fn控制激活函数
with tf.name_scope('loss'):
    xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss=tf.reduce_mean(xentropy,name='loss')
    #sparse_..,利用softmax激活函数然后计算交叉熵；softmax_cross_entropy_with_logits
    #则将标签进行one-hot编码
learning_rate=0.01
with tf.name_scope('train'):
    optimizer=tf.train.GradientDescentOptimizer(learning_rate)
    training_op=optimizer.minimize(loss)

#定义评估模型的标准，
with tf.name_scope('eval'):
    correct=tf.nn.in_top_k(logits,y,1)#最高的logit是否对应目标类
    #返回一个1维布尔tensor
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))#将布尔tensor变成浮点数并计算均值
    #即得的其精度
    
init=tf.global_variables_initializer()
saver=tf.train.Saver()

from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('/data/')

n_epochs=400
batch_size=50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for i in range(mnist.train.num_examples//batch_size):
            X_batch,y_batch=mnist.train.next_batch(batch_size)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
        acc_train=accuracy.eval(feed_dict={X:X_batch,y:y_batch})
        #在最后一个mini-batch和整个训练集上计算训练精度
        acc_test=accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
        print(epoch,"Train accuracy:",acc_train,'Test accuracy:',acc_test)
    save_path=saver.save(sess,'./my_model_final.ckpt')
    


Extracting /data/train-images-idx3-ubyte.gz
Extracting /data/train-labels-idx1-ubyte.gz
Extracting /data/t10k-images-idx3-ubyte.gz
Extracting /data/t10k-labels-idx1-ubyte.gz
0 Train accuracy: 0.9 Test accuracy: 0.9025
1 Train accuracy: 0.92 Test accuracy: 0.9179
2 Train accuracy: 0.92 Test accuracy: 0.9292
3 Train accuracy: 0.96 Test accuracy: 0.9349
4 Train accuracy: 1.0 Test accuracy: 0.9414
5 Train accuracy: 0.98 Test accuracy: 0.9459
6 Train accuracy: 0.96 Test accuracy: 0.9489
7 Train accuracy: 1.0 Test accuracy: 0.9529
8 Train accuracy: 0.96 Test accuracy: 0.9534
9 Train accuracy: 0.96 Test accuracy: 0.9568
10 Train accuracy: 0.98 Test accuracy: 0.9586
11 Train accuracy: 0.98 Test accuracy: 0.9603
12 Train accuracy: 1.0 Test accuracy: 0.9624
13 Train accuracy: 0.96 Test accuracy: 0.9635
14 Train accuracy: 0.98 Test accuracy: 0.9643
15 Train accuracy: 0.96 Test accuracy: 0.9659
16 Train accuracy: 1.0 Test accuracy: 0.9659
17 Train accuracy: 1.0 Test accuracy: 0.9673
18 Train accur

177 Train accuracy: 1.0 Test accuracy: 0.978
178 Train accuracy: 1.0 Test accuracy: 0.9784
179 Train accuracy: 1.0 Test accuracy: 0.9783
180 Train accuracy: 1.0 Test accuracy: 0.9784
181 Train accuracy: 1.0 Test accuracy: 0.978
182 Train accuracy: 1.0 Test accuracy: 0.9785
183 Train accuracy: 1.0 Test accuracy: 0.9782
184 Train accuracy: 1.0 Test accuracy: 0.9783
185 Train accuracy: 1.0 Test accuracy: 0.9785
186 Train accuracy: 1.0 Test accuracy: 0.9783
187 Train accuracy: 1.0 Test accuracy: 0.9783
188 Train accuracy: 1.0 Test accuracy: 0.9782
189 Train accuracy: 1.0 Test accuracy: 0.9781
190 Train accuracy: 1.0 Test accuracy: 0.9785
191 Train accuracy: 1.0 Test accuracy: 0.9782
192 Train accuracy: 1.0 Test accuracy: 0.9784
193 Train accuracy: 1.0 Test accuracy: 0.9784
194 Train accuracy: 1.0 Test accuracy: 0.9782
195 Train accuracy: 1.0 Test accuracy: 0.9783
196 Train accuracy: 1.0 Test accuracy: 0.9782
197 Train accuracy: 1.0 Test accuracy: 0.9783
198 Train accuracy: 1.0 Test accurac

356 Train accuracy: 1.0 Test accuracy: 0.9784
357 Train accuracy: 1.0 Test accuracy: 0.9784
358 Train accuracy: 1.0 Test accuracy: 0.9782
359 Train accuracy: 1.0 Test accuracy: 0.9785
360 Train accuracy: 1.0 Test accuracy: 0.9784
361 Train accuracy: 1.0 Test accuracy: 0.9783
362 Train accuracy: 1.0 Test accuracy: 0.9783
363 Train accuracy: 1.0 Test accuracy: 0.9784
364 Train accuracy: 1.0 Test accuracy: 0.9785
365 Train accuracy: 1.0 Test accuracy: 0.9784
366 Train accuracy: 1.0 Test accuracy: 0.9785
367 Train accuracy: 1.0 Test accuracy: 0.9783
368 Train accuracy: 1.0 Test accuracy: 0.9784
369 Train accuracy: 1.0 Test accuracy: 0.9784
370 Train accuracy: 1.0 Test accuracy: 0.9784
371 Train accuracy: 1.0 Test accuracy: 0.9781
372 Train accuracy: 1.0 Test accuracy: 0.9783
373 Train accuracy: 1.0 Test accuracy: 0.9785
374 Train accuracy: 1.0 Test accuracy: 0.9784
375 Train accuracy: 1.0 Test accuracy: 0.9784
376 Train accuracy: 1.0 Test accuracy: 0.9784
377 Train accuracy: 1.0 Test accur

In [ ]:
with tf.Session() as sess:
    saver.restore(sess,'./my_model_final.ckpt')
    X_new_scaled=[...]#some new scaled image
    Z=logits.eval(feed_dict={X:X_new_scaled})
    y_pred=np.argmax(Z,axis=1)